In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!pip install tensorflow-gpu

In [ ]:
pip install geemap==0.15.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 2.1 MB/s 
     |████████████████████████████████| 198 kB 57.1 MB/s 
     |████████████████████████████████| 55 kB 4.3 MB/s 
     |████████████████████████████████| 46 kB 3.8 MB/s 
     |████████████████████████████████| 3.4 MB 40.1 MB/s 
     |████████████████████████████████| 1.3 MB 46.0 MB/s 
     |████████████████████████████████| 8.8 MB 39.2 MB/s 
     |████████████████████████████████| 130 kB 69.3 MB/s 
     |████████████████████████████████| 3.0 MB 32.2 MB/s 
     |████████████████████████████████| 98 kB 8.0 MB/s 
     |████████████████████████████████| 99 kB 9.4 MB/s 
     |████████████████████████████████| 1.2 MB 45.0 MB/s 
     |████████████████████████████████| 128 kB 56.3 MB/s 
     |████████████████████████████████| 1.6 MB 41.8 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 9.

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.8.2


In [ ]:
pip list

In [ ]:
nvidia-smi

In [ ]:
pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
torch.cuda.device_count()

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
!pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!apt-get install nvidia-384

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
import geemap
Map = geemap.Map(center=[40,-100], zoom=4)
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=5m3g8Vu5zQTdXB6ftSqXDAixYNgIkKCa9w8KP9iFkCU&tc=CAYIgZdc-VIKoUvFYUwKMYmksl_o2DwSNMmx8kMv5zA&cc=aKRGBlR4FUoFC7ZNj4V0TKdImCF2KZCtpWZRQU-jDCQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJoGwLoStFOEo3ucv20pWsCrd4yn-yZtHNaED01yab2ry_29nDMyMKo

Successfully saved authorization token.


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
import ee
data = Map.user_roi
# region = ee.FeatureCollection('users/ruanyongjian123/Yangtze_Plain_merge')
# region_geo = region.geometry()
region = ee.FeatureCollection('users/ruanyongjian123/Yangtze_Plain_merge')
region_geo = region.geometry()
span = 8.9831528*0.2
fishnet = geemap.fishnet(region_geo, h_interval=span, v_interval=span, delta=1)
Map.addLayer(fishnet, {}, 'ROI')
Map.addLayer(region_geo, {}, 'ROI2')

In [ ]:
import os
def get_img(asc_mosaic, aoi_arr, nums_aoi, time_sign):
  task_arr = []
  for i in range(nums_aoi):
    tmp_name = str(i) + "_fishnet"
    new_dir = "/content/drive/MyDrive/" + time_sign
    if os.path.exists(new_dir)==False:
      os.makedirs(new_dir)
    feature = aoi_arr.get(i)
    feature2 = ee.Feature(feature)
    region_geo = feature2.geometry()
    image_task = ee.batch.Export.image.toDrive(
    image = asc_mosaic,
    description=time_sign + "_" + tmp_name,
    fileNamePrefix=time_sign + "_" + tmp_name,
    folder = time_sign,
    scale=10,
    region=region_geo,
    maxPixels = 1e13,
    # fileDimensions=[25600, 25600]
    )
    task_arr.append(image_task)
  return task_arr

In [ ]:
nums_aoi = fishnet.size().getInfo()
aoi_arr = fishnet.toList(nums_aoi)
print(nums_aoi)
sentinel_dataset = ee.ImageCollection("COPERNICUS/S1_GRD").filter(ee.Filter.date('2022-09-11','2022-09-22')).filterBounds(region_geo).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).select("VV").mosaic()
sentinel_dataset = sentinel_dataset.toFloat()
task_arr = get_img(sentinel_dataset, aoi_arr, nums_aoi, "VV_0911-0922")
for task in task_arr:
  task.start()


41


In [ ]:
import os
import time
region = ee.FeatureCollection('users/ruanyongjian123/Yangtze_Plain_merge')
region_geo = region.geometry()
# nums_aoi = region.size().getInfo()
# aoi_arr = region.toList(nums_aoi)
# aa = aoi_arr.get(0)
# ff = aa.geometry()
# print(ff)
# sentinel_dataset = ee.ImageCollection('COPERNICUS/S1_GRD').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.date('2022-08-22','2022-09-10')).filterBounds(region_geo)
sentinel_dataset = ee.ImageCollection("COPERNICUS/S1_GRD").filter(ee.Filter.date('2022-09-11','2022-09-22')).filterBounds(region_geo).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).select("VH").mosaic()
dirs = "/content/drive/MyDrive/out"
out_dirs = "/content/drive/MyDrive/poyang"
os.makedirs("/content/drive/MyDrive/VH_0911-0922")
image_task = ee.batch.Export.image.toDrive(
    image = sentinel_dataset,
    description='20220911-0922',
    fileNamePrefix='VH_0911-0922',
    folder = 'VH_0911-0922',
    scale=10,
    region=region_geo,
    maxPixels = 1e13,
    # fileDimensions=[25600, 25600]
  )
image_task.start()
print('Running image export to Cloud Storage...')
import time

while image_task.active():
  time.sleep(30)
  if image_task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')
    break

# task_arr = get_img(asc_mosaic, aoi_arr, nums_aoi)

# for task in task_arr:
#   task.start()

# while True:
#   status_arr = []
#   for task in task_arr:
#     if task.active():

#     if task.status()['state'] != 'COMPLETED':
#       print('Error with image export.')
#     else:
#       print('Image export completed')
#     status_arr.append(task.status()['state'])
#     time.sleep(30)

  # dirs = "/content/drive/MyDrive/out"
  # out_dirs = "/content/drive/MyDrive/poyang"
  # image_task = ee.batch.Export.image.toDrive(
  #   image = asc_vh_mosaic,
  #   description='poyang',
  #   fileNamePrefix='poyang_data2',
  #   folder = 'poyang',
  #   scale=10,
  #   region=region,
  #   maxPixels = 1e12,
  #   fileDimensions=[25600, 25600]
  # )

# image_task.start()
# print('Running image export to Cloud Storage...')
# import time

# while image_task.active():
#   L = len(os.listdir(out_dirs))
#   if L % 5 == 0:
#     print(L)
#   time.sleep(30)

# if image_task.status()['state'] != 'COMPLETED':
#   print('Error with image export.')
# else:
#   print('Image export completed.')


In [ ]:
import os
import time
region = ee.FeatureCollection('users/ruanyongjian123/Yangtze_Plain_merge')
region_geo = region.geometry()
# nums_aoi = region.size().getInfo()
# aoi_arr = region.toList(nums_aoi)
# aa = aoi_arr.get(0)
# ff = aa.geometry()
# print(ff)
# sentinel_dataset = ee.ImageCollection('COPERNICUS/S1_GRD').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.date('2022-08-22','2022-09-10')).filterBounds(region_geo)
sentinel_dataset = ee.ImageCollection("COPERNICUS/S1_GRD").filter(ee.Filter.date('2022-09-11','2022-09-22')).filterBounds(region_geo).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).select("VV").mosaic()
dirs = "/content/drive/MyDrive/out"
out_dirs = "/content/drive/MyDrive/poyang"
os.makedirs("/content/drive/MyDrive/VV_0911-0922")
image_task = ee.batch.Export.image.toDrive(
    image = sentinel_dataset,
    description='20220911-0922',
    fileNamePrefix='VV_0911-0922',
    folder = 'VV_0911-0922',
    scale=10,
    region=region_geo,
    maxPixels = 1e13,
    # fileDimensions=[25600, 25600]
  )
image_task.start()
print('Running image export to Cloud Storage...')
import time

while image_task.active():
  time.sleep(30)
  if image_task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')
    break

In [ ]:
import os
import glob
dirs = "/content/drive/MyDrive/VH_0911-0922"
out_patch_dir = "/content/drive/MyDrive/VH_0911-0922_patch"
os.makedirs(out_patch_dir)
tif_files = glob.glob(dirs + "/*.tif")

In [ ]:
import os
import glob
dirs = "/content/drive/MyDrive/VH_0911-0922_patch"
tif_files = glob.glob(dirs + "/*.tif")


18081


In [ ]:
from osgeo import ogr, osr, gdal
import numpy as np
def gen_data(filename, out_dir, size = 1000):
  in_ds = gdal.Open(filename)
  print("open tif file succeed")
  width = in_ds.RasterXSize
  height = in_ds.RasterYSize
  outbandsize = in_ds.RasterCount
  im_geotrans = in_ds.GetGeoTransform()
  im_proj = in_ds.GetProjection()
  datatype = in_ds.GetRasterBand(1).DataType
  in_band1 = in_ds.GetRasterBand(1)
  offset_x = 0
  offset_y = 0

  col_num = int(width / size)
  row_num = int(height / size)
  if(width % size != 0):
      col_num += 1
  if(height % size != 0):
      row_num += 1

  num = 1
  tmp = filename.split("/")[-1]
  tmp_name = tmp.split(".")[0]
  print(filename, "row_num:%d   col_num:%d" %(row_num,col_num))
  for i in range(row_num):
      for j in range(col_num):
          offset_x = i * size
          offset_y = j * size
          b_ysize = min(width - offset_y, size)
          b_xsize = min(height - offset_x, size)
          files = out_dir + "/" + tmp_name + "_" + str(i) + "_" + str(j) +"_.tif"
          if os.path.exists(files):
            continue
          out_band1 = in_band1.ReadAsArray(offset_y, offset_x, b_ysize, b_xsize)
          gtif_driver = gdal.GetDriverByName("GTiff")

          if num % 10 == 0:
            print('total nums:', row_num * col_num, "remain:", row_num * col_num - num)
          out_ds = gtif_driver.Create(files, b_ysize, b_xsize, outbandsize, datatype)

          ori_transform = in_ds.GetGeoTransform()

          top_left_x = ori_transform[0]
          w_e_pixel_resolution = ori_transform[1]
          top_left_y = ori_transform[3]
          n_s_pixel_resolution = ori_transform[5]

          top_left_x = top_left_x + offset_x * w_e_pixel_resolution
          top_left_y = top_left_y + offset_y * n_s_pixel_resolution
          dst_transform = (top_left_x, ori_transform[1], ori_transform[2], top_left_y, ori_transform[4], ori_transform[5])
          out_ds.SetGeoTransform(dst_transform)
          out_ds.SetProjection(in_ds.GetProjection())
          out_ds.GetRasterBand(1).WriteArray(out_band1)
          out_ds.FlushCache()
          num+=1
          del out_ds,out_band1


In [ ]:
import os
import glob
dirs = "/content/drive/MyDrive/VH_0911-0922"
out_patch_dir = "/content/drive/MyDrive/VH_0911-0922_patch"
tif_files = glob.glob(dirs + "/*.tif")
for files in tif_files:
  gen_data(files, out_patch_dir)


In [ ]:
import os
import argparse
import numpy as np
import os,time,cv2
import numpy as np
import scipy.io as sio
import tifffile as tiff
import gdal
from osgeo import gdal_array
import gdalnumeric
from osgeo import ogr, osr, gdal
from PIL import Image, ImageDraw


def get_data(filename):
    in_ds = gdal.Open(filename)
    print("open tif file succeed")
    width = in_ds.RasterXSize
    height = in_ds.RasterYSize
    outbandsize = in_ds.RasterCount
    im_geotrans = in_ds.GetGeoTransform()
    im_proj = in_ds.GetProjection()
    datatype = in_ds.GetRasterBand(1).DataType
    return in_ds, width, height, outbandsize, datatype, im_geotrans, im_proj

def clip_data_sar(im_data, width, height, outbandsize, datatype, out_dir, size):
    count = 0
    in_band1 = im_data.GetRasterBand(1)

    col_num = int(width / size) #宽度可以分成几块
    row_num = int(height / size)#高度可以分成几块

    num = 0

    print("row_num:%d   col_num:%d" %(row_num,col_num))
    for i in range(row_num):    #从高度下手！！！ 可以分成几块！
        for j in range(col_num):
            offset_x = j * size
            offset_y = i * size

            b_ysize = size #min(width - offset_y, size)
            b_xsize = size #min(height - offset_x, size)

            out_band1 = in_band1.ReadAsArray(offset_x, offset_y, b_ysize, b_xsize)


            gtif_driver = gdal.GetDriverByName("GTiff")
            file = out_dir + "/" + str(i) + "_" + str(j) +"_.tiff"
            num += 1
            out_ds = gtif_driver.Create(file, b_ysize, b_xsize, outbandsize, datatype)
            ori_transform = im_data.GetGeoTransform()

            top_left_x = ori_transform[0]  # 左上角x坐标
            w_e_pixel_resolution = ori_transform[1]  # 东西方向像素分辨率
            top_left_y = ori_transform[3]  # 左上角y坐标
            n_s_pixel_resolution = ori_transform[5]  # 南北方向像素分辨率

            top_left_x = top_left_x + offset_x * w_e_pixel_resolution
            top_left_y = top_left_y + offset_y * n_s_pixel_resolution

            dst_transform = (top_left_x, ori_transform[1], ori_transform[2], top_left_y, ori_transform[4], ori_transform[5])
            out_ds.SetGeoTransform(dst_transform)

            out_ds.GetRasterBand(1).WriteArray(out_band1)

            out_ds.FlushCache()
            del out_ds, out_band1

    print(count)

sar_data, width, height, outbandsize, datatype, im_geotrans, im_proj  = get_data(r"/content/drive/MyDrive/poyang/poyang_data2.tif")
clip_data_sar(sar_data, width, height, outbandsize, datatype, "/content/drive/MyDrive/out", size = 1000)


In [ ]:
import os
import time
import glob
import cv2
import numpy as np
from skimage import segmentation

import torch
import torch.nn as nn
import tifffile as tiff
from skimage.util import img_as_float

import gdal

def get_data(filename):
  in_ds = gdal.Open(filename)
  width = in_ds.RasterXSize  #
  height = in_ds.RasterYSize  #
  outbandsize = in_ds.RasterCount  #
  im_geotrans = in_ds.GetGeoTransform()  #
  im_proj = in_ds.GetProjection()  #
  datatype = in_ds.GetRasterBand(1).DataType
  return in_ds, width, height, outbandsize, datatype, im_geotrans, im_proj

def sample_wise_standardization(data):
  import math
  _mean = np.mean(data)
  _std = np.std(data)
  npixel = np.size(data) * 1.0
  min_stddev = 1.0 / math.sqrt(npixel)
  return (data - _mean) / max(_std, min_stddev)


class Args(object):
  input_image_path = 'image/aerial.jpg'  # image/coral.jpg image/tiger.jpg
  train_epoch = 2 ** 7
  mod_dim1 = 16  #
  mod_dim2 = 16
  gpu_id = 0

  min_label_num = 5  # if the label number small than it, break loop
  max_label_num = 256  # if the label number small than it, start to show result image.


class MyNet(nn.Module):
  def __init__(self, inp_dim, mod_dim1, mod_dim2):
    super(MyNet, self).__init__()

    self.seq = nn.Sequential(
        nn.Conv2d(inp_dim, mod_dim1, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(mod_dim1),
        nn.ReLU(inplace=True),

        nn.Conv2d(mod_dim1, mod_dim2, kernel_size=1, stride=1, padding=0),
        nn.BatchNorm2d(mod_dim2),
        nn.ReLU(inplace=True),

        nn.Conv2d(mod_dim2, mod_dim1, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(mod_dim1),
        nn.ReLU(inplace=True),

        nn.Conv2d(mod_dim1, mod_dim2, kernel_size=1, stride=1, padding=0),
        nn.BatchNorm2d(mod_dim2),
    )
  def forward(self, x):
    return self.seq(x)


def run(filename, dirs):
  args = Args()
  torch.cuda.manual_seed_all(1943)
  np.random.seed(1943)
  os.environ['CUDA_VISIBLE_DEVICES'] = str(args.gpu_id)  # choose GPU:0
  tmp_ = filename.split("/")
  fff = dirs + "/" + tmp_[-1]
  if os.path.exists(fff):
    return
  image = tiff.imread(filename)
  h, w = image.shape
  if h!= 1000 or w!=1000:
    return
  image[image != image] = -15

  image = img_as_float(image[:, :])
  image = (image - np.min(image)) / (np.max(image) - np.min(image)) * 255
  image = image[:, :]
  image = image.astype(np.uint8)

  '''segmentation ML'''
  # seg_map = segmentation.felzenszwalb(image, scale=32, sigma=0.5, min_size=64)
  #seg_map = segmentation.felzenszwalb(image, scale=100, sigma=0.5, min_size=32)
  seg_map = segmentation.felzenszwalb(image, scale=100, sigma=0.5, min_size=16)
  #seg_map = segmentation.felzenszwalb(image)
  # seg_map = segmentation.slic(image, n_segments=10000, compactness=100)
  seg_map = seg_map.flatten()
  seg_lab = [np.where(seg_map == u_label)[0]
              for u_label in np.unique(seg_map)]

  '''train init'''
  device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

  tensor = sample_wise_standardization(image)
  tensor = np.expand_dims(tensor, axis = 2)

  tensor = tensor.transpose((2, 0, 1))
  tensor = tensor.astype(np.float32) / 255.0
  tensor = tensor[np.newaxis, :, :, :]
  tensor = torch.from_numpy(tensor).to(device)

  model = MyNet(inp_dim=1, mod_dim1=args.mod_dim1, mod_dim2=args.mod_dim2).to(device)
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.9)
    # optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-1, momentum=0.0)

  image_flatten = image.reshape((-1, 1))
  color_avg = np.random.randint(255, size=(args.max_label_num, 1))
  show = image
  model.train()
  for batch_idx in range(args.train_epoch):
    '''forward'''
    optimizer.zero_grad()
    output = model(tensor)[0]
    output = output.permute(1, 2, 0).view(-1, args.mod_dim2)
    target = torch.argmax(output, 1)
    im_target = target.data.cpu().numpy()

    '''refine'''
    for inds in seg_lab:
      u_labels, hist = np.unique(im_target[inds], return_counts=True)
      im_target[inds] = u_labels[np.argmax(hist)]

    target = torch.from_numpy(im_target)
    target = target.to(device)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    un_label, lab_inverse = np.unique(im_target, return_inverse=True, )
    if un_label.shape[0] < args.max_label_num:  # update show
      img_flatten = image_flatten.copy()
      if len(color_avg) != un_label.shape[0]:
        color_avg = [np.mean(img_flatten[im_target == label], axis=0, dtype=np.int) for label in un_label]
      for lab_id, color in enumerate(color_avg):
        img_flatten[lab_inverse == lab_id] = color
      show = img_flatten.reshape(image.shape)
    tmp2 = filename.split("/")
    if len(un_label) < args.min_label_num:
      break
  tmp22 = filename.split("/")
  tiff.imwrite(dirs + "/" + tmp22[-1], show)

dirs = "/content/drive/MyDrive/VH_0911-0922_patch"
tif_files = glob.glob(dirs + "/*.tif")
new_dirs = "/content/drive/MyDrive/VH_0911-0922_patch_seg"
num = 0
for filename in tif_files:
  run(filename, new_dirs)
  if num % 100 ==0:
    print(filename)
    num += 1

/content/drive/MyDrive/VH_0911-0922_patch/VH_0911-0922_38_fishnet_15_8_.tif


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in true_divide


# New Section

In [ ]:
import os
import time
import glob
import cv2
import numpy as np
from skimage import segmentation

import torch
import torch.nn as nn
import tifffile as tiff
from skimage.util import img_as_float

import gdal

def get_data(filename):
    in_ds = gdal.Open(filename)
    # print("open tif file succeed")
    width = in_ds.RasterXSize  #
    height = in_ds.RasterYSize  #
    outbandsize = in_ds.RasterCount  #
    im_geotrans = in_ds.GetGeoTransform()  #
    im_proj = in_ds.GetProjection()  #
    datatype = in_ds.GetRasterBand(1).DataType
    return in_ds, width, height, outbandsize, datatype, im_geotrans, im_proj


def sample_wise_standardization(data):


    import math
    _mean = np.mean(data)
    _std = np.std(data)
    npixel = np.size(data) * 1.0
    min_stddev = 1.0 / math.sqrt(npixel)
    return (data - _mean) / max(_std, min_stddev)


class Args(object):
    input_image_path = 'image/aerial.jpg'  # image/coral.jpg image/tiger.jpg
    train_epoch = 2 ** 7
    mod_dim1 = 16  #
    mod_dim2 = 16
    gpu_id = 0

    min_label_num = 5  # if the label number small than it, break loop
    max_label_num = 256  # if the label number small than it, start to show result image.


class MyNet(nn.Module):
    def __init__(self, inp_dim, mod_dim1, mod_dim2):
        super(MyNet, self).__init__()

        self.seq = nn.Sequential(
            nn.Conv2d(inp_dim, mod_dim1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(mod_dim1),
            nn.ReLU(inplace=True),

            nn.Conv2d(mod_dim1, mod_dim2, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(mod_dim2),
            nn.ReLU(inplace=True),

            nn.Conv2d(mod_dim2, mod_dim1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(mod_dim1),
            nn.ReLU(inplace=True),

            nn.Conv2d(mod_dim1, mod_dim2, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(mod_dim2),
        )

    def forward(self, x):
        return self.seq(x)


def run(filename, dirs):
    # world, width, height, outbandsize, datatype, im_geotrans, im_proj = get_data(filename)

   # start_time0 = time.time()

    args = Args()
    torch.cuda.manual_seed_all(1943)
    np.random.seed(1943)
    os.environ['CUDA_VISIBLE_DEVICES'] = str(args.gpu_id)  # choose GPU:0

    image = tiff.imread(filename)
    image[image != image] = -15

    image = img_as_float(image[:, :])
    image = (image - np.min(image)) / (np.max(image) - np.min(image)) * 255
    image = image[:, :]
    image = image.astype(np.uint8)

    '''segmentation ML'''
    # seg_map = segmentation.felzenszwalb(image, scale=32, sigma=0.5, min_size=64)
    #seg_map = segmentation.felzenszwalb(image, scale=100, sigma=0.5, min_size=32)
    seg_map = segmentation.felzenszwalb(image, scale=100, sigma=0.5, min_size=16)
    #seg_map = segmentation.felzenszwalb(image)
    # seg_map = segmentation.slic(image, n_segments=10000, compactness=100)
    seg_map = seg_map.flatten()
    seg_lab = [np.where(seg_map == u_label)[0]
               for u_label in np.unique(seg_map)]

    '''train init'''
    device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

    tensor = sample_wise_standardization(image)
    tensor = np.expand_dims(tensor, axis = 2)

    tensor = tensor.transpose((2, 0, 1))
    tensor = tensor.astype(np.float32) / 255.0
    tensor = tensor[np.newaxis, :, :, :]
    tensor = torch.from_numpy(tensor).to(device)

    model = MyNet(inp_dim=1, mod_dim1=args.mod_dim1, mod_dim2=args.mod_dim2).to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.9)
    # optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-1, momentum=0.0)

    image_flatten = image.reshape((-1, 1))
    color_avg = np.random.randint(255, size=(args.max_label_num, 1))
    show = image

    '''train loop'''
    #start_time1 = time.time()
    model.train()
    for batch_idx in range(args.train_epoch):
        '''forward'''
        optimizer.zero_grad()
        output = model(tensor)[0]
        output = output.permute(1, 2, 0).view(-1, args.mod_dim2)
        target = torch.argmax(output, 1)
        im_target = target.data.cpu().numpy()

        '''refine'''
        for inds in seg_lab:
            u_labels, hist = np.unique(im_target[inds], return_counts=True)
            im_target[inds] = u_labels[np.argmax(hist)]

        '''backward'''
        target = torch.from_numpy(im_target)
        target = target.to(device)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        '''show image'''
        un_label, lab_inverse = np.unique(im_target, return_inverse=True, )
        if un_label.shape[0] < args.max_label_num:  # update show
            img_flatten = image_flatten.copy()
            if len(color_avg) != un_label.shape[0]:
                color_avg = [np.mean(img_flatten[im_target == label], axis=0, dtype=np.int) for label in un_label]
            for lab_id, color in enumerate(color_avg):
                img_flatten[lab_inverse == lab_id] = color
            show = img_flatten.reshape(image.shape)
        # cv2.imshow("seg_pt", show)
        tmp2 = filename.split("/")
        # ttt = show.astype(np.uint8)
        # if batch_idx % 15 == 0:
        #     cv2.imwrite(dirs + "/" + str(batch_idx) + "_" + tmp2[-1], ttt)
        #cv2.waitKey(1)

       # print('Loss:', batch_idx, loss.item())
        if len(un_label) < args.min_label_num:
            break
    tmp22 = filename.split("/")
    tiff.imwrite(dirs + "/" + tmp22[-1], show)


if __name__ == '__main__':
    dirs = "/content/drive/MyDrive/VH_0911-0922_patch"
    tif_files = glob.glob(dirs + "/*.tif")
    new_dirs = "/content/drive/MyDrive/VH_0911-0922_patch_seg"
    # os.makedirs(new_dirs)
    num = 0
    for filename in tif_files:
        run(filename, new_dirs)
        num += 1
        if num % 100 ==0:
            print(num)


In [ ]:
!nvidia-smi

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

try:
  sess.run(tf.global_variables_initializer())
  print('ok')
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')

Instructions for updating:
non-resource variables are not supported in the long term


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/convolutional.py:414: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


ok


In [ ]:

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save("/content/drive/MyDrive/mnist.h5")

11501568/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


AttributeError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')  # 将google硬盘挂载在/comtent/drive/目录上面

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
